In [7]:
from plasma_classes import *
from plasma_utils import *
from matplotlib import pyplot as plt
from matplotlib import animation
from tqdm import tqdm
import math

ModuleNotFoundError: No module named 'plasma_classes'

In [8]:
def run_simulation(const_params, varying_params):
    # Unpack constant parameters
    L = const_params['L']
    N_x = const_params['N_x']
    N_p = const_params['N_p']
    h = L / N_x
    tau = const_params['tau']
    n0 = const_params['n0']
    n1 = n0 * N_x / N_p
    q = const_params['q']
    m_e = const_params['m_e']
    m_i = const_params['m_i']
    epsilon = const_params['epsilon']
    T_i = const_params['T_i']
    k_b = const_params['k_b']
    
    # Initialize empty lists to store results
    temp_ion_fluxes = []
    temp_electron_fluxes = []
    temp_sum_fluxes = []
    temps = []

    temp_values = range(varying_params["temp_min"], varying_params["temp_max"], varying_params["temp_step"])
    # Loop over varying parameter values
    for temp in temp_values:
        T_e = temp
        temps.append(T_e)
        v_t_e = math.sqrt(3 * k_b * T_e / m_e)
        vmin_e = -3 * v_t_e
        vmax_e = 3 * v_t_e

        # Create particles and nodes objects
        ions = Particles(N_p, n1, q, m_i)
        electrons = Particles(N_p, n1, -q, m_e)
        nodes = Nodes(N_x)

        # Set initial particle distributions
        set_homogeneous(electrons, 0, L)
        set_homogeneous(ions, 0, L)
        set_distr(electrons, Maxwell(T_e, k_b, m_e), vmin_e, vmax_e, 1000)
        set_distr(ions, Maxwell(T_i, k_b, m_i), -vmax_e, vmax_e, 1000)
        electrons.normalise(h, tau)
        ions.normalise(h, tau)

        # Run simulation
        n = varying_params['num_iterations']
        for t in tqdm(range(n)):
            try:
                move(electrons, nodes, consistency=True)
                move(ions, nodes, consistency=True)
            except Exception:
                print("number of iteration: ", t)
                break

            nodes.rho *= 0
            account_walls(nodes, electrons, const_params['left_wall'], const_params['right_wall'])
            account_walls(nodes, ions, const_params['left_wall'], const_params['right_wall'])
            get_rho(nodes, electrons)
            get_rho(nodes, ions)

            calc_fields(nodes, h, epsilon, periodic=True)
            accel(electrons, nodes, L, h, tau)
            accel(ions, nodes, L, h, tau)

            electrons.denormalise(h, tau)
            ions.denormalise(h, tau)
            electrons.normalise(h, tau)
            ions.normalise(h, tau)

        temp_electron_fluxes.append(N_p - electrons.n_macro)
        temp_ion_fluxes.append(N_p - ions.n_macro)
        
    # Compute sum fluxes
    for i in range(len(temps)):
        temp_sum_fluxes.append(temp_electron_fluxes[i] + temp_ion_fluxes[i])

    # Return results
    return {'temp_values': temps,
            'electron_fluxes': temp_electron_fluxes,
            'ion_fluxes': temp_ion_fluxes,
            'sum_fluxes': temp_sum_fluxes}


In [9]:
# Define the constant simulation parameters
sim_params = {"L": 1e-2, "N_x": 200, "N_p": 5000, "tau": 1e-13, "n0": 1e17, "q": 1.60217e-19,
              "m_e": 9.1093837e-31, "m_i": 1.672e-27, "epsilon": 8.85e-12, "T_i": 600, "k_b": 1.38e-23}

# Define the varying temperature dictionary
temp_dict = {"temp_min": 11600, "temp_max": 11600*2, "temp_step": 11600}

# Call the function and get the fluxes
temp_ion_fluxes, temp_electron_fluxes, temp_sum_fluxes = run_simulation(sim_params, temp_dict)


NameError: name 'math' is not defined